In [1]:
from sklearn.datasets import fetch_openml
mnist=fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]

import matplotlib as mpl
import matplotlib.pyplot as plt
some_digit = X[0]
#X[0]:in last note book we saw that it is 5
some_digit_image = some_digit.reshape(28, 28)


import numpy as np
y = y.astype(np.uint8)

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [2]:
#only try to identify one digit for example 5
#a binary classifier, capable of distinguishing between just two classes, 5 and not-5
y_train_5 = (y_train == 5)  # True for all 5s, False other digits
y_test_5 = (y_test == 5)


In [3]:
# Stochastic Gradient Descent (SGD) classifier
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [4]:
sgd_clf.predict([some_digit])

array([ True])

In [6]:
# evaluate this model’s performance ------> Using Cross-Validation(using K-fold cross-validation)---> k=3
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")

array([0.95035, 0.96035, 0.9604 ])

In [7]:
#a very dumb classifier that just classifies every single image in the “not-5” class!!!
from sklearn.base import BaseEstimator
class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        return self
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)
    
never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring="accuracy")
#because only about 10% of the images are 5s, so if you always guess that an image is not a 5, you will be right 
#about 90% of the time

           # so accuracy is generally not the preferred performance measure for classifiers
#especially when you are dealing with skewed datasets (i.e., when some classes are much more frequent than others)

array([0.91125, 0.90855, 0.90915])

In [8]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

In [11]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pred)

array([[53892,   687],
       [ 1891,  3530]])

In [12]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_train_5, y_train_pred)
#When it claims an image represents a 5, it is correct only 0.83 of the time

0.8370879772350012

In [13]:
recall_score(y_train_5, y_train_pred)
#it only detects 0.65 of the 5s.

0.6511713705958311

In [14]:
from sklearn.metrics import f1_score
f1_score(y_train_5, y_train_pred)

0.7325171197343846